# Gemini Chatbot with Gradio Interface

In [ ]:
%pip install -q -U google-generativeai
%pip install -q gradio

In [ ]:
import gradio as gr
import google.generativeai as genai
import os

# IMPORTANT: Replace with your actual Gemini API Key
GOOGLE_API_KEY = "Enter Your Gemini API Key Here"

if not GOOGLE_API_KEY or GOOGLE_API_KEY == "Enter Your Gemini API Key Here":
    print("Please replace 'Enter Your Gemini API Key Here' with your actual key.")
    model = None
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    try:
        model = genai.GenerativeModel('gemini-1.5-flash')
        print("Gemini model initialized successfully.")
    except Exception as e:
        print(f"Error initializing Gemini model: {e}")
        model = None

CSS = """
/* Dark Theme Overrides */
body, .gradio-container { background-color: #111827; color: #f3f4f6; font-family: 'Inter', sans-serif; }
#sidebar {
    background-color: #111827;
    color: white;
    padding: 1rem;
    min-width: 260px;
    height: 100vh;
    display: flex;
    flex-direction: column;
}
#sidebar_header { display: flex; align-items: center; margin-bottom: 2rem; }
#sidebar_header .icon {
    width: 2rem; height: 2rem; background-color: #10b981; border-radius: 9999px;
    display: flex; align-items: center; justify-content: center; margin-right: 0.5rem;
}
#sidebar_header .title { font-weight: bold; font-size: 1.125rem; }
#new-chat-btn {
    display: flex; align-items: center; width: 100%; padding: 0.75rem;
    border-radius: 0.375rem; border: 1px solid #374151;
    background-color: #111827; color: white; margin-bottom: 1rem;
    cursor: pointer;
}
#new-chat-btn:hover { background-color: #1f2937; }
#sidebar-content { flex: 1; overflow-y: auto; }
#sidebar-footer { margin-top: auto; padding-top: 1rem; border-top: 1px solid #374151; }
.sidebar-link { display: flex; align-items: center; padding: 0.75rem; border-radius: 0.375rem; color: white; }
.sidebar-link:hover { background-color: #1f2937; cursor: pointer; }
.sidebar-link span { white-space: nowrap; overflow: hidden; text-overflow: ellipsis; }
#chat_container { padding: 1rem; flex-grow: 1; }
#chatbot {
    height: calc(100vh - 200px) !important;
    overflow-y: auto;
    border: none;
    box-shadow: none;
    background-color: transparent;
}
.message-bubble {
    padding: 0.75rem 1rem !important;
    border-radius: 0.5rem !important;
    box-shadow: 0 1px 2px 0 rgba(0, 0, 0, 0.05) !important;
    max-width: 80% !important;
    word-wrap: break-word !important;
}
.message-bubble.user {
    background-color: #10b981 !important;
    color: #ffffff !important;
    align-self: flex-end;
}
.message-bubble.bot {
    background-color: #374151 !important;
    color: #f3f4f6 !important;
    align-self: flex-start;
}
#initial-view { text-align: center; margin: 2rem auto; max-width: 48rem; }
#initial-view .icon {
    width: 4rem; height: 4rem; background-color: #374151; border-radius: 9999px;
    display: flex; align-items: center; justify-content: center; margin: 0 auto 1rem;
}
#initial-view h1 { font-size: 1.5rem; font-weight: bold; margin-bottom: 0.5rem; color: #f3f4f6;}
.prompt-grid {
    display: grid; grid-template-columns: repeat(2, minmax(0, 1fr));
    gap: 1rem; margin-top: 3rem;
}
.prompt-card {
    background-color: #1f2937; padding: 1rem; border-radius: 0.5rem;
    border: 1px solid #374151; text-align: left; cursor: pointer;
}
.prompt-card:hover { border-color: #4b5563; }
.prompt-card h3 { font-weight: 600; margin-bottom: 0.5rem; color: #f3f4f6; }
.prompt-card p { color: #9ca3af; font-size: 0.875rem; }
#input-area {
    background-color: #111827;
    padding: 1rem;
    border-top: 1px solid #374151;
}
#input-container {
    max-width: 40rem;
    margin: 0 auto;
    position: relative;
}
#msg_textbox, #msg_textbox textarea {
    background-color: #1f2937 !important;
    color: white !important;
    padding-right: 11rem !important;
    border-radius: 0.5rem !important;
    border: 1px solid #4b5563 !important;
}
#send_button {
    position: absolute !important;
    right: 0.5rem;
    bottom: 0.5rem;
    width: 2rem;
    height: 2rem;
    background-color: #10b981 !important;
    color: white !important;
    border-radius: 0.375rem !important;
    font-size: 0.9rem;
}
#send_button:hover { background-color: #059669 !important; }
#history-container .gradio-button {
    background-color: transparent !important;
    border: none !important;
    text-align: left !important;
    padding: 0.75rem !important;
    border-radius: 0.375rem !important;
    color: white !important;
    display: block !important;
    width: 100% !important;
}
#history-container .gradio-button:hover {
    background-color: #1f2937 !important;
}
#context-menu {
    position: absolute;
    z-index: 1000;
    width: 150px;
    background-color: #1f2937;
    border: 1px solid #374151;
    border-radius: 0.5rem;
    padding: 0.5rem;
    display: none;
}
#context-menu button {
    width: 100%;
    background: none;
    border: none;
    color: white;
    text-align: left;
    padding: 0.5rem;
    border-radius: 0.25rem;
}
#context-menu button:hover {
    background-color: #374151;
}
"""

def convert_history_for_api(history):
    api_history = []
    for msg in history:
        content = msg.get('content', '')
        if content is None:
            content = ''
        role = 'model' if msg['role'] == 'assistant' else msg['role']
        api_history.append({'role': role, 'parts': [{'text': content}]})
    return api_history

def chat_response_stream(history):
    if not history or len(history) < 2:
        return

    user_message = history[-2]['content']
    
    if not model:
        history[-1]['content'] = "Error: Model not initialized."
        yield history
        return

    api_history = convert_history_for_api(history[:-2])
    session = model.start_chat(history=api_history)

    try:
        response = session.send_message(user_message, stream=True)
        full_response = ""
        for chunk in response:
            full_response += chunk.text
            history[-1]['content'] = full_response
            yield history
    except Exception as e:
        history[-1]['content'] = f"An error occurred: {e}"
        yield history

js_code = """
function() {
    gradioApp().then(() => {
        function handlePromptClick(text) {
            const msgTextbox = document.querySelector('#msg_textbox textarea');
            const sendButton = document.querySelector('#send_button');
            if (msgTextbox && sendButton) {
                msgTextbox.value = text;
                const inputEvent = new Event('input', { bubbles: true });
                msgTextbox.dispatchEvent(inputEvent);
                sendButton.click();
            }
        }

        let currentContextMenuIndex = -1;
        const menu = document.getElementById('context-menu');

        const appContainer = document.querySelector('.gradio-container');
        appContainer.addEventListener('contextmenu', function(e) {
            const historyButton = e.target.closest('[id^="history-btn-"]');
            if (historyButton) {
                e.preventDefault();
                currentContextMenuIndex = parseInt(historyButton.id.split('-')[2]);
                menu.style.display = 'block';
                menu.style.left = e.pageX + 'px';
                menu.style.top = e.pageY + 'px';
            }
        });

        appContainer.addEventListener('click', function(e) {
            if (menu) {
                menu.style.display = 'none';
            }
        });
        
        const renameBtn = document.getElementById('rename-chat');
        if (renameBtn) {
            renameBtn.addEventListener('click', function() {
                const newName = prompt('Enter new chat name:');
                if (newName && newName.trim() !== '') {
                    const indexInput = document.querySelector('#rename-index-input textarea');
                    const nameInput = document.querySelector('#new-name-input textarea');
                    const triggerBtn = document.querySelector('#rename-trigger-btn');
                    
                    indexInput.value = currentContextMenuIndex;
                    nameInput.value = newName;
                    
                    indexInput.dispatchEvent(new Event('input', { bubbles: true }));
                    nameInput.dispatchEvent(new Event('input', { bubbles: true }));
                    
                    triggerBtn.click();
                }
            });
        }

        const deleteBtn = document.getElementById('delete-chat');
        if (deleteBtn) {
            deleteBtn.addEventListener('click', function() {
                if (confirm('Are you sure you want to delete this chat?')) {
                    const indexInput = document.querySelector('#delete-index-input textarea');
                    const triggerBtn = document.querySelector('#delete-trigger-btn');
                    
                    indexInput.value = currentContextMenuIndex;
                    indexInput.dispatchEvent(new Event('input', { bubbles: true }));
                    triggerBtn.click();
                }
            });
        }

        const initialView = document.getElementById('initial-view');
        if(initialView) {
            initialView.addEventListener('click', (e) => {
                const card = e.target.closest('.prompt-card');
                if (card) {
                    const text = card.querySelector('h3').textContent + ' ' + card.querySelector('p').textContent;
                    handlePromptClick(text);
                }
            });
        }
    });
}
"""

with gr.Blocks(css=CSS, theme=gr.themes.Base(), js=js_code) as demo:
    all_chats = gr.State([])

    with gr.Row():
        with gr.Column(elem_id="sidebar", scale=1):
            with gr.Row(elem_id="sidebar_header"):
                gr.HTML('<div class="icon"><i class="fas fa-comment-dots"></i></div><span class="title">Gemini Chat</span>', visible=True)
            new_chat_btn = gr.Button("＋ New chat", elem_id="new-chat-btn")
            with gr.Column(elem_id="history-container"):
                gr.Markdown("### Chat History")
                history_buttons = [gr.Button(visible=False, elem_id=f"history-btn-{i}") for i in range(10)]

        with gr.Column(elem_id="chat_container", scale=4):
            with gr.Column(elem_id="initial-view") as initial_view:
                 gr.HTML("""
                    <div class="icon" style="margin-top: 2rem;"><i class="fas fa-comment-dots text-green-400 text-2xl"></i></div>
                    <h1>How can I help you today?</h1>
                    <div class="prompt-grid">
                        <div id="prompt-card-1" class="prompt-card"><h3>Explain quantum computing</h3><p>in simple terms</p></div>
                        <div id="prompt-card-2" class="prompt-card"><h3>Got any creative ideas</h3><p>for a 10 year old's birthday?</p></div>
                        <div id="prompt-card-3" class="prompt-card"><h3>How do I make an HTTP request</h3><p>in JavaScript?</p></div>
                        <div id="prompt-card-4" class="prompt-card"><h3>Write a poem</h3><p>about artificial intelligence</p></div>
                    </div>
                 """, visible=True)

            chatbot = gr.Chatbot(
                elem_id="chatbot",
                type="messages",
                avatar_images=(None, "https://i.imgur.com/g5g6Qhf.png"),
                visible=False
            )
            
            with gr.Row(elem_id="input-container"):
                msg = gr.Textbox(
                    elem_id="msg_textbox",
                    show_label=False,
                    placeholder="Message Gemini...",
                    lines=1,
                    scale=8
                )
                send_btn = gr.Button("➤", elem_id="send_button", scale=1)
    
    with gr.Row(visible=False):
        rename_index_input = gr.Textbox(elem_id="rename-index-input")
        new_name_input = gr.Textbox(elem_id="new-name-input")
        rename_trigger_btn = gr.Button(elem_id="rename-trigger-btn")
        delete_index_input = gr.Textbox(elem_id="delete-index-input")
        delete_trigger_btn = gr.Button(elem_id="delete-trigger-btn")

    gr.HTML("""
        <div id="context-menu">
            <button id="rename-chat">Rename</button>
            <button id="delete-chat">Delete</button>
        </div>
    """)


    def handle_user_message(message, history):
        if not message or not message.strip():
            return history
        
        history.append({'role': 'user', 'content': message})
        history.append({'role': 'assistant', 'content': ''})
        return history

    def show_chat_and_clear_textbox(history):
        return {
            initial_view: gr.update(visible=False),
            chatbot: gr.update(visible=True, value=history),
            msg: gr.update(value="")
        }

    def save_and_clear_session(current_history, all_history):
        if current_history and len(current_history) > 1:
            title = current_history[0]['content'][:40]
            all_history.insert(0, {'title': title, 'history': current_history})
            if len(all_history) > 10: all_history.pop()
        
        history_updates = []
        for i in range(10):
            if i < len(all_history):
                history_updates.append(gr.update(value=all_history[i]['title'], visible=True))
            else:
                history_updates.append(gr.update(visible=False))

        return (
            all_history,
            gr.update(value=[], visible=False),
            gr.update(visible=True),
            *history_updates
        )

    def save_current_then_load_chat(current_history, all_history, index):
        if current_history and len(current_history) > 1:
            is_saved = any(chat['history'] == current_history for chat in all_history)
            if not is_saved:
                title = current_history[0]['content'][:40]
                all_history.insert(0, {'title': title, 'history': current_history})
                if len(all_history) > 10: all_history.pop()
                if index != -1: index += 1

        chat_to_load = all_history[index]
        
        history_updates = []
        for i in range(10):
            if i < len(all_history):
                history_updates.append(gr.update(value=all_history[i]['title'], visible=True))
            else:
                history_updates.append(gr.update(visible=False))
        
        return (
            all_history,
            gr.update(value=chat_to_load['history'], visible=True),
            gr.update(visible=False),
            *history_updates
        )

    def rename_chat(all_history, index_str, new_name):
        index = int(index_str)
        if 0 <= index < len(all_history):
            all_history[index]['title'] = new_name
        
        history_updates = []
        for i in range(10):
            if i < len(all_history):
                history_updates.append(gr.update(value=all_history[i]['title'], visible=True))
            else:
                history_updates.append(gr.update(visible=False))
        return (all_history, *history_updates)

    def delete_chat(all_history, index_str, current_chat):
        index = int(index_str)
        chatbot_update = gr.update()
        initial_view_update = gr.update()

        if 0 <= index < len(all_history):
            if current_chat == all_history[index]['history']:
                chatbot_update = gr.update(value=[], visible=False)
                initial_view_update = gr.update(visible=True)
            all_history.pop(index)
        
        history_updates = []
        for i in range(10):
            if i < len(all_history):
                history_updates.append(gr.update(value=all_history[i]['title'], visible=True))
            else:
                history_updates.append(gr.update(visible=False))

        return (all_history, chatbot_update, initial_view_update, *history_updates)

    # Event chain for text input
    msg.submit(
        handle_user_message,
        [msg, chatbot],
        [chatbot],
        queue=False
    ).then(
        show_chat_and_clear_textbox,
        [chatbot],
        [initial_view, chatbot, msg]
    ).then(
        chat_response_stream,
        [chatbot],
        [chatbot]
    )

    send_btn.click(
        handle_user_message,
        [msg, chatbot],
        [chatbot],
        queue=False
    ).then(
        show_chat_and_clear_textbox,
        [chatbot],
        [initial_view, chatbot, msg]
    ).then(
        chat_response_stream,
        [chatbot],
        [chatbot]
    )

    new_chat_btn.click(
        save_and_clear_session,
        [chatbot, all_chats],
        [all_chats, chatbot, initial_view] + history_buttons
    )

    for i, btn in enumerate(history_buttons):
        btn.click(
            save_current_then_load_chat,
            inputs=[chatbot, all_chats, gr.State(i)],
            outputs=[all_chats, chatbot, initial_view] + history_buttons
        )

    rename_trigger_btn.click(
        rename_chat,
        [all_chats, rename_index_input, new_name_input],
        [all_chats] + history_buttons
    )

    delete_trigger_btn.click(
        delete_chat,
        [all_chats, delete_index_input, chatbot],
        [all_chats, chatbot, initial_view] + history_buttons
    )

if __name__ == "__main__":
    demo.head = '<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">'
    demo.queue().launch(debug=True)
